# Setup

In [ ]:
# Docker and Milvus Standalone

# 1. wget https://github.com/milvus-io/milvus/releases/download/v2.0.2/milvus-standalone-docker-compose.yml -O docker-compose.yml
# 2. https://docs.docker.com/compose/install/
# 3. Start: sudo docker-compose up -d 
#     Check:sudo docker-compose ps
#     Stop: sudo docker-compose down
#     Delete data after stop: sudo rm -rf  volumes


In [2]:
# Python 3.7.1 or later is required

In [ ]:
# pymilvus

# python3 -m pip install pymilvus==2.0.2
#  Verify: python3 -c "from pymilvus import Collection"

# Import Libraries

In [6]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)
import time
import numpy as np

# Read Embeddings

### This notebook was run on EC2 (Deep Learning AMI) to connect to Milvus servers for the purpose of vector search. Input to this notebook can be found here: https://drive.google.com/drive/folders/1M4_SrEPMRo9PSNLWzPHPHN0MpPD4uzjB?usp=sharing

In [3]:
import os

In [7]:
X_train_features = np.loadtxt(os.path.join("0004",'X_train_features.txt'))
X_test_features = np.loadtxt(os.path.join("0004",'X_test_features.txt'))

In [8]:
num_entities, dim = X_train_features.shape

In [9]:
y_train = np.loadtxt(os.path.join("0004",'y_train.txt'))
y_test = np.loadtxt(os.path.join("0004",'y_test.txt'))

In [10]:
train_images = np.loadtxt(os.path.join("0004",'train_images.txt'))
test_images = np.loadtxt(os.path.join("0004",'test_images.txt'))

In [11]:
X_train_features.shape

(10586, 1792)

In [12]:
y_train.astype(int)

array([5543,  896,  886, ..., 2310,  802,  385])

# Connect to Server

In [11]:
connections.connect("default", host="localhost", port="19530")

# Create Collection

In [13]:
has = utility.has_collection("facenet")
print(f"Does collection facenet exist in Milvus: {has}")

Does collection facenet exist in Milvus: False


In [17]:
fmt = "\n=== {:30} ===\n"

In [18]:
fields = [
    FieldSchema(name="pk", dtype=DataType.INT64,
                is_primary=True, auto_id=False, description="primary id"),
    FieldSchema(name="label", dtype=DataType.INT64, description="label"),
    FieldSchema(name="image", dtype=DataType.INT64, description="image_number"),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR,
                dim=dim, description="vector")
]

schema = CollectionSchema(fields, "Embeddings for facenet")
print(fmt.format("Create collection `embed`"))
embed = Collection("embed", schema,
                          using='default', shards_num=2)


=== Create collection `embed`      ===



In [24]:
print(fmt.format("Start inserting entities"))

entities = [
    [i for i in range(0, num_entities)],  # field pk
    y_train.astype(int), # labels
    train_images.astype(int), # images
    X_train_features,  # field embeddings
]
insert_result = embed.insert(entities)


=== Start inserting entities       ===



In [33]:
print(entities[3])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [25]:
print(f"Number of Entities: {embed.num_entities}")

Number of Entities: 10586


# Build Index on Entities

In [37]:
import math

print(fmt.format("Start Creating index IVF_FLAT"))

# optimal value of nlist is 4 * sqrt(num of enteries)
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": int(4 * math.sqrt(num_entities))},
}
embed.create_index("embeddings", index)


=== Start Creating index IVF_FLAT  ===



Status(code=0, message='')

# Load to Memory

In [39]:
print(fmt.format("Start loading"))

embed.load()


=== Start loading                  ===



# Vector Similarity Search

## Similartiy Search

In [40]:
# nprobe / nlist = 1/4
search_params = {
    "metric_type": "l2",
    "params": {"nprobe": 330},
}

In [87]:
start_time = time.time()
# here limit is set to 7 to get 7 similar images later will be chaged to 20
result = embed.search(X_test_features, "embeddings", search_params, limit=7, output_fields=["label", "image", "pk"])
end_time = time.time()

In [88]:
print(f'Time Taken: {end_time - start_time}s')

Time Taken: 5.631507158279419s


# Result 

In [89]:
final_result = dict()

# final_result = {
#     'label_imgNumber': {
#         "label_imgNumber": distance,
#         "label_imgNumber": distance,
#         "label_imgNumber": distance,
        
#     },
#     ...
# }

In [90]:
y_test

array([ 173., 4836., 1428., ..., 5354., 3109., 1393.])

In [91]:
test_images

array([ 2., 26.,  2., ...,  1.,  3., 41.])

In [92]:
def get_query_number(number):
    temp = str(number)
    while len(temp) != 4:
        temp = '0' + temp
    return temp

In [93]:
index = 0
for hits in result:
    print(f'Query Number: {index}')
    
    query_label = int(y_test[index])
    query_imgNumber = int(test_images[index])
    
    current_query_id = str(query_label) + '_' + get_query_number(query_imgNumber)
    
    final_result[current_query_id] = dict()
    
    for hit in hits:
        result_label = int(hit.entity.get('label'))
        result_imgNumber = int(hit.entity.get('image'))
        
        current_result_id = str(result_label) + '_' + get_query_number(result_imgNumber)
        
        final_result[current_query_id][current_result_id] = hit.distance
        
        print(f"hit: {hit}, Label: {hit.entity.get('label')}, Image: {hit.entity.get('image')}")
    
    index += 1

Query Number: 0
hit: (distance: 26.057170867919922, id: 119), Label: 173, Image: 1
hit: (distance: 127.43841552734375, id: 1957), Label: 999, Image: 1
hit: (distance: 130.5911865234375, id: 7793), Label: 1105, Image: 2
hit: (distance: 130.891357421875, id: 8090), Label: 2368, Image: 1
hit: (distance: 131.17236328125, id: 4305), Label: 1610, Image: 1
hit: (distance: 135.7631378173828, id: 3508), Label: 3250, Image: 1
hit: (distance: 136.7291259765625, id: 3623), Label: 2368, Image: 2
Query Number: 1
hit: (distance: 37.43260192871094, id: 4727), Label: 4836, Image: 8
hit: (distance: 53.6215934753418, id: 6419), Label: 4836, Image: 13
hit: (distance: 59.88230895996094, id: 3374), Label: 4836, Image: 19
hit: (distance: 60.98465347290039, id: 10396), Label: 4836, Image: 5
hit: (distance: 63.946720123291016, id: 6971), Label: 4836, Image: 9
hit: (distance: 66.75064086914062, id: 3127), Label: 4836, Image: 23
hit: (distance: 67.14411926269531, id: 5197), Label: 4836, Image: 17
Query Number: 2

hit: (distance: 233.34690856933594, id: 6413), Label: 3467, Image: 5
Query Number: 126
hit: (distance: 87.2083511352539, id: 6170), Label: 2075, Image: 4
hit: (distance: 89.87103271484375, id: 5747), Label: 2075, Image: 3
hit: (distance: 97.59793853759766, id: 8776), Label: 2075, Image: 2
hit: (distance: 152.41738891601562, id: 8079), Label: 2878, Image: 1
hit: (distance: 163.1081085205078, id: 7988), Label: 3212, Image: 1
hit: (distance: 163.1358642578125, id: 9915), Label: 4062, Image: 1
hit: (distance: 166.01821899414062, id: 7886), Label: 3335, Image: 3
Query Number: 127
hit: (distance: 65.844482421875, id: 3267), Label: 5458, Image: 131
hit: (distance: 77.38188171386719, id: 5088), Label: 5458, Image: 38
hit: (distance: 78.87896728515625, id: 7206), Label: 5458, Image: 25
hit: (distance: 79.29905700683594, id: 6590), Label: 5458, Image: 4
hit: (distance: 83.17660522460938, id: 9302), Label: 5458, Image: 40
hit: (distance: 84.04142761230469, id: 4982), Label: 5458, Image: 33
hit: (

hit: (distance: 150.10186767578125, id: 8414), Label: 444, Image: 1
hit: (distance: 158.92657470703125, id: 2094), Label: 584, Image: 1
hit: (distance: 161.87489318847656, id: 998), Label: 2901, Image: 1
hit: (distance: 164.2852020263672, id: 2253), Label: 444, Image: 2
hit: (distance: 167.12033081054688, id: 832), Label: 4898, Image: 1
hit: (distance: 173.80010986328125, id: 5427), Label: 3411, Image: 3
Query Number: 156
hit: (distance: 66.10835266113281, id: 584), Label: 4114, Image: 5
hit: (distance: 90.63130950927734, id: 6401), Label: 4114, Image: 2
hit: (distance: 92.84676361083984, id: 4252), Label: 4114, Image: 8
hit: (distance: 96.28128051757812, id: 8828), Label: 4114, Image: 13
hit: (distance: 99.16423797607422, id: 1248), Label: 4114, Image: 11
hit: (distance: 102.51397705078125, id: 8212), Label: 4114, Image: 7
hit: (distance: 114.55485534667969, id: 6059), Label: 4114, Image: 12
Query Number: 157
hit: (distance: 105.675537109375, id: 617), Label: 2500, Image: 18
hit: (dis

Query Number: 284
hit: (distance: 38.23125076293945, id: 5902), Label: 417, Image: 3
hit: (distance: 38.57796859741211, id: 962), Label: 417, Image: 12
hit: (distance: 49.595787048339844, id: 6594), Label: 417, Image: 5
hit: (distance: 53.27894592285156, id: 8172), Label: 417, Image: 2
hit: (distance: 56.886817932128906, id: 9269), Label: 417, Image: 22
hit: (distance: 59.8803825378418, id: 7366), Label: 417, Image: 1
hit: (distance: 74.66339111328125, id: 8577), Label: 417, Image: 7
Query Number: 285
hit: (distance: 125.2592544555664, id: 8409), Label: 4859, Image: 1
hit: (distance: 134.16921997070312, id: 4040), Label: 1091, Image: 2
hit: (distance: 135.94174194335938, id: 6743), Label: 4672, Image: 2
hit: (distance: 139.41748046875, id: 651), Label: 3995, Image: 5
hit: (distance: 143.574462890625, id: 4722), Label: 3995, Image: 3
hit: (distance: 145.25311279296875, id: 10193), Label: 2891, Image: 1
hit: (distance: 147.46571350097656, id: 4960), Label: 779, Image: 1
Query Number: 286

hit: (distance: 131.66421508789062, id: 6514), Label: 3318, Image: 1
hit: (distance: 134.99391174316406, id: 4409), Label: 423, Image: 1
hit: (distance: 157.23651123046875, id: 3567), Label: 1338, Image: 2
hit: (distance: 159.94357299804688, id: 6569), Label: 229, Image: 1
hit: (distance: 173.4142303466797, id: 3233), Label: 2637, Image: 1
hit: (distance: 174.1541748046875, id: 7460), Label: 1114, Image: 1
hit: (distance: 175.2963409423828, id: 4655), Label: 5264, Image: 1
Query Number: 314
hit: (distance: 116.52391052246094, id: 4225), Label: 4182, Image: 1
hit: (distance: 117.1749267578125, id: 4544), Label: 5694, Image: 1
hit: (distance: 117.57991027832031, id: 2128), Label: 3104, Image: 1
hit: (distance: 124.37317657470703, id: 642), Label: 5700, Image: 1
hit: (distance: 125.62275695800781, id: 7427), Label: 2186, Image: 4
hit: (distance: 125.86576843261719, id: 7974), Label: 4566, Image: 1
hit: (distance: 127.35232543945312, id: 1188), Label: 5608, Image: 2
Query Number: 315
hit: 

...

hit: (distance: 53.76569366455078, id: 39), Label: 5458, Image: 101
hit: (distance: 55.82567596435547, id: 9118), Label: 5458, Image: 107
hit: (distance: 62.71601104736328, id: 2273), Label: 5458, Image: 117
hit: (distance: 64.4134750366211, id: 697), Label: 5458, Image: 55
hit: (distance: 70.15971374511719, id: 8942), Label: 5458, Image: 64
hit: (distance: 70.99058532714844, id: 6538), Label: 5458, Image: 22
Query Number: 2239
hit: (distance: 151.85818481445312, id: 2940), Label: 2821, Image: 2
hit: (distance: 156.79226684570312, id: 5034), Label: 194, Image: 1
hit: (distance: 157.78453063964844, id: 1077), Label: 2051, Image: 1
hit: (distance: 160.34701538085938, id: 10329), Label: 3875, Image: 1
hit: (distance: 160.97433471679688, id: 1251), Label: 4336, Image: 1
hit: (distance: 168.19183349609375, id: 5415), Label: 2191, Image: 1
hit: (distance: 169.5654296875, id: 8260), Label: 5091, Image: 2
Query Number: 2240
hit: (distance: 64.13137817382812, id: 8352), Label: 5600, Image: 2
hi

hit: (distance: 24.878719329833984, id: 6746), Label: 5250, Image: 2
hit: (distance: 38.103424072265625, id: 10094), Label: 5250, Image: 5
hit: (distance: 42.74183654785156, id: 6090), Label: 5250, Image: 3
hit: (distance: 42.87854766845703, id: 2050), Label: 5250, Image: 8
hit: (distance: 46.649635314941406, id: 2890), Label: 5250, Image: 6
hit: (distance: 49.992950439453125, id: 3672), Label: 5250, Image: 10
hit: (distance: 50.5277214050293, id: 1953), Label: 5250, Image: 11
Query Number: 2271
hit: (distance: 148.23748779296875, id: 5065), Label: 4110, Image: 1
hit: (distance: 148.91542053222656, id: 1468), Label: 4335, Image: 1
hit: (distance: 150.93972778320312, id: 3616), Label: 4109, Image: 1
hit: (distance: 151.89212036132812, id: 9488), Label: 926, Image: 1
hit: (distance: 152.89291381835938, id: 1171), Label: 3900, Image: 2
hit: (distance: 156.52658081054688, id: 752), Label: 2573, Image: 1
hit: (distance: 157.52584838867188, id: 583), Label: 3900, Image: 1
Query Number: 2272


hit: (distance: 119.24394989013672, id: 4420), Label: 4686, Image: 6
hit: (distance: 147.2813720703125, id: 3374), Label: 4836, Image: 19
Query Number: 2429
hit: (distance: 140.57350158691406, id: 2868), Label: 189, Image: 6
hit: (distance: 144.88746643066406, id: 3823), Label: 2182, Image: 1
hit: (distance: 146.2556610107422, id: 3864), Label: 4865, Image: 1
hit: (distance: 146.52056884765625, id: 4636), Label: 4974, Image: 1
hit: (distance: 152.66009521484375, id: 8615), Label: 784, Image: 2
hit: (distance: 154.24276733398438, id: 615), Label: 189, Image: 1
hit: (distance: 158.81069946289062, id: 4089), Label: 17, Image: 4
Query Number: 2430
hit: (distance: 147.1808624267578, id: 1424), Label: 2895, Image: 1
hit: (distance: 153.26461791992188, id: 3013), Label: 4770, Image: 1
hit: (distance: 165.1093292236328, id: 5705), Label: 1992, Image: 2
hit: (distance: 166.0082244873047, id: 8555), Label: 4615, Image: 1
hit: (distance: 168.76205444335938, id: 5663), Label: 3462, Image: 2
hit: (

Query Number: 2565
hit: (distance: 107.87322998046875, id: 2160), Label: 813, Image: 1
hit: (distance: 144.3643035888672, id: 6027), Label: 1059, Image: 4
hit: (distance: 153.66317749023438, id: 7829), Label: 1059, Image: 5
hit: (distance: 155.93553161621094, id: 9161), Label: 368, Image: 1
hit: (distance: 158.04290771484375, id: 7208), Label: 1059, Image: 2
hit: (distance: 159.048095703125, id: 643), Label: 469, Image: 1
hit: (distance: 161.23162841796875, id: 9136), Label: 1059, Image: 6
Query Number: 2566
hit: (distance: 47.93610382080078, id: 3698), Label: 3270, Image: 5
hit: (distance: 52.0937614440918, id: 10236), Label: 3270, Image: 8
hit: (distance: 54.315853118896484, id: 9293), Label: 3270, Image: 4
hit: (distance: 74.51268005371094, id: 8530), Label: 3270, Image: 1
hit: (distance: 77.12777709960938, id: 6071), Label: 3270, Image: 7
hit: (distance: 87.8106689453125, id: 3691), Label: 3270, Image: 9
hit: (distance: 167.8284454345703, id: 1331), Label: 3892, Image: 3
Query Numb

hit: (distance: 40.08051300048828, id: 6575), Label: 3104, Image: 7
hit: (distance: 53.9455680847168, id: 3045), Label: 3104, Image: 6
hit: (distance: 73.26724243164062, id: 9971), Label: 3104, Image: 4
Query Number: 2599
hit: (distance: 67.03400421142578, id: 8699), Label: 1870, Image: 466
hit: (distance: 71.30258178710938, id: 8191), Label: 1870, Image: 355
hit: (distance: 72.0323715209961, id: 9664), Label: 1870, Image: 467
hit: (distance: 72.23062896728516, id: 2629), Label: 1870, Image: 184
hit: (distance: 74.91110229492188, id: 163), Label: 1870, Image: 194
hit: (distance: 75.728515625, id: 2484), Label: 1870, Image: 515
hit: (distance: 75.73306274414062, id: 1415), Label: 1870, Image: 431
Query Number: 2600
hit: (distance: 162.55735778808594, id: 9066), Label: 3559, Image: 1
hit: (distance: 165.92922973632812, id: 3410), Label: 653, Image: 1
hit: (distance: 166.97763061523438, id: 7933), Label: 3238, Image: 2
hit: (distance: 171.34396362304688, id: 7746), Label: 963, Image: 1
hi

In [94]:
final_result

{'173_0002': {'173_0001': 26.057170867919922,
  '999_0001': 127.43841552734375,
  '1105_0002': 130.5911865234375,
  '2368_0001': 130.891357421875,
  '1610_0001': 131.17236328125,
  '3250_0001': 135.7631378173828,
  '2368_0002': 136.7291259765625},
 '4836_0026': {'4836_0008': 37.43260192871094,
  '4836_0013': 53.6215934753418,
  '4836_0019': 59.88230895996094,
  '4836_0005': 60.98465347290039,
  '4836_0009': 63.946720123291016,
  '4836_0023': 66.75064086914062,
  '4836_0017': 67.14411926269531},
 '1428_0002': {'1428_0001': 117.2981948852539,
  '4769_0001': 136.78451538085938,
  '5299_0001': 152.08587646484375,
  '3125_0001': 159.51443481445312,
  '2579_0001': 164.4651336669922,
  '4348_0001': 165.86683654785156,
  '2661_0003': 166.17874145507812},
 '5367_0016': {'5367_0010': 87.85360717773438,
  '5367_0014': 92.42774200439453,
  '5367_0002': 120.11233520507812,
  '5367_0017': 129.45809936523438,
  '5367_0009': 129.7062530517578,
  '5367_0012': 132.98597717285156,
  '5367_0004': 133.9378

In [95]:
import json

# Serialize data into file - to store the output for visualization purposes
json.dump( final_result, open( "result_seven.json", 'w' ) )

In [98]:
# Read data from file:
data = json.load( open( "result_seven.json" ) )

In [99]:
data

{'173_0002': {'173_0001': 26.057170867919922,
  '999_0001': 127.43841552734375,
  '1105_0002': 130.5911865234375,
  '2368_0001': 130.891357421875,
  '1610_0001': 131.17236328125,
  '3250_0001': 135.7631378173828,
  '2368_0002': 136.7291259765625},
 '4836_0026': {'4836_0008': 37.43260192871094,
  '4836_0013': 53.6215934753418,
  '4836_0019': 59.88230895996094,
  '4836_0005': 60.98465347290039,
  '4836_0009': 63.946720123291016,
  '4836_0023': 66.75064086914062,
  '4836_0017': 67.14411926269531},
 '1428_0002': {'1428_0001': 117.2981948852539,
  '4769_0001': 136.78451538085938,
  '5299_0001': 152.08587646484375,
  '3125_0001': 159.51443481445312,
  '2579_0001': 164.4651336669922,
  '4348_0001': 165.86683654785156,
  '2661_0003': 166.17874145507812},
 '5367_0016': {'5367_0010': 87.85360717773438,
  '5367_0014': 92.42774200439453,
  '5367_0002': 120.11233520507812,
  '5367_0017': 129.45809936523438,
  '5367_0009': 129.7062530517578,
  '5367_0012': 132.98597717285156,
  '5367_0004': 133.9378

# Realse Collection

In [101]:
utility.drop_collection("embed") # Not Important but to drop the collection we just created

In [ ]:
# python3 hello_milvus.py

# Accuracy

In [14]:
import json
import os
from sklearn.metrics import accuracy_score

In [4]:
data = json.load( open(os.path.join("0004_milvus", "result_three.json")))

In [15]:
y_test.astype(int)

array([ 173, 4836, 1428, ..., 5354, 3109, 1393])

In [27]:
# load the saved outputs to calculate accuracy
y_pred = list()
for index, result in data.items():
    
    image = list()
    label = list()
    dist = list()
    
    for result_index, result_dist in result.items():
        label.append(result_index.split('_')[0])
        image.append(result_index.split('_')[1])
        dist.append(result_dist)
    y_pred.append(int(max(label,key=label.count)))
    print(image, label, dist, sep='\n')

y_pred = np.array(y_pred)

['0001', '0001', '0002']
['173', '999', '1105']
[26.057170867919922, 127.43841552734375, 130.5911865234375]
['0008', '0013', '0019']
['4836', '4836', '4836']
[37.43260192871094, 53.6215934753418, 59.88230895996094]
['0001', '0001', '0001']
['1428', '4769', '5299']
[117.2981948852539, 136.78451538085938, 152.08587646484375]
['0010', '0014', '0002']
['5367', '5367', '5367']
[87.85360717773438, 92.42774200439453, 120.11233520507812]
['0024', '0002', '0002']
['3401', '3914', '4380']
[148.79164123535156, 155.47116088867188, 156.21974182128906]
['0003', '0002', '0002']
['4631', '4631', '1966']
[154.74160766601562, 155.43798828125, 162.2541046142578]
['0004', '0001', '0002']
['5258', '5258', '5258']
[55.15113067626953, 60.07026290893555, 61.48064422607422]
['0001', '0002', '0002']
['4882', '4882', '3929']
[167.69810485839844, 183.77108764648438, 201.8341522216797]
['0001', '0010', '0002']
['4003', '4001', '4003']
[147.05587768554688, 153.54180908203125, 157.07298278808594]
['0002', '0001', '0

[175.0430908203125, 182.35047912597656, 191.53187561035156]
['0004', '0007', '0005']
['1696', '1696', '1696']
[80.97842407226562, 105.23661804199219, 108.58224487304688]
['0001', '0009', '0007']
['5400', '5400', '5400']
[92.28768157958984, 98.13172149658203, 99.55165100097656]
['0014', '0013', '0024']
['386', '386', '386']
[47.165191650390625, 77.46882629394531, 80.70758819580078]
['0006', '0008', '0003']
['5219', '5219', '5219']
[71.21333312988281, 105.11368560791016, 134.889892578125]
['0011', '0008', '0001']
['4972', '4972', '4972']
[102.13230895996094, 116.90585327148438, 124.04317474365234]
['0004', '0002', '0002']
['4228', '4228', '1508']
[86.03629302978516, 92.91939544677734, 189.09503173828125]
['0005', '0004', '0002']
['4911', '4911', '1558']
[101.09368896484375, 147.58554077148438, 162.4354248046875]
['0001', '0002', '0001']
['3469', '14', '5686']
[128.21644592285156, 155.35308837890625, 161.3409423828125]
['0002', '0001', '0001']
['1476', '4961', '3355']
[136.8139190673828, 

In [28]:
accuracy_score(y_test, y_pred)

0.6218360408009067

### Here you can see that we received an accuracy of 62%. Given that LFW have many people with 2 or less images this is good and much better than our previous approaches.